In [5]:
import ndjson
import numpy as np

In [6]:
split = ("train", "test", "val")
anno = {}
sim = {}

for s in split:
    with open(f"moma/{s}.ndjson", "r") as f:
        anno[s] = ndjson.load(f)
    sim[s] = np.load(f"moma/sm_dtw_{s}.npy")

print(f"sim matrix shape: {sim['train'].shape}")
print(f"annotation length: {len(anno['train'])}")

sim matrix shape: (903, 903)
annotation length: 903


In [7]:
threshold = 0.3

pos = {}
neg = {}
for s in split:
    pos_inds = []
    neg_inds = []
    for idx in range(len(anno[s])):
        ind = np.argsort(sim[s][idx])[::-1]
        median_len = int(len(anno[s]) * threshold)
        pos_inds.append(ind[:median_len])
        neg_inds.append(ind[-median_len:])
    pos_inds = np.stack(pos_inds)
    neg_inds = np.stack(neg_inds)
    pos[s] = pos_inds
    neg[s] = neg_inds


In [8]:
pos['train'].shape

(903, 270)

In [9]:
for s, arr in pos.items():
    np.save(f"moma/triplet/{s}_pos_thr={threshold}.npy", arr)

for s, arr in neg.items():
    np.save(f"moma/triplet/{s}_neg_thr={threshold}.npy", arr)